<a href="https://colab.research.google.com/github/AlpinDale/misc-scripts/blob/main/Aphrodite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to the Aphrodite Engine Colab!

If you're on mobile, please tab on the play button below. If you're not, you can safely skip it and go to the next cell.

The default model is Mythalion-13B, but you can either type in your model name, or select one of the defaults in the dropdown.

If you're running a non-quantized model, please set the Quantization to `None` instead. AWQ and GPTQ quantization is also available, but you will need Colab Pro with better GPUs.


In [ ]:
#@title <b>v-- Tap this if you play on Mobile</b> { display-mode: "form" }
%%html
<b>Press play on the music player to keep the tab alive, then start KoboldAI below (You can ignore this step if you used run all and are on PC)</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

In [ ]:
#@title <b>v-- The free plan on Google Colab only supports up to 13B (in GPTQ/AWQ) formats.</b>
#@markdown The free plan on Google Colab only supports up to 13B (in GPTQ/AWQ) formats.
#@markdown You can enter a custom model as well (for models bigger than 13B GPTQ/AWQ, you will need Colab Pro).

Model = "TheBloke/Mythalion-13B-AWQ" #@param ["TheBloke/Mythalion-13B-AWQ", "TheBloke/MythoMax-L2-13B-AWQ", "TheBloke/Pygmalion-2-13B-AWQ", "TheBloke/MLewd-L2-Chat-13B-AWQ"]{allow-input: true}
Quantization = "awq" #@param ['awq', 'gptq', "None"]
!pip uninstall aphrodite-engine -y
!pip install aphrodite-engine
!wget -c https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!nohup ./cloudflared-linux-amd64 tunnel --url http://127.0.0.1:2242 &
!sleep 10
!cat nohup.out
!python -m aphrodite.endpoints.kobold.api_server --model $Model --dtype float16 --host 127.0.0.1 -q $Quantization --gpu-memory-utilization 0.88 &
